<a href="https://colab.research.google.com/github/woo-jungnam/TH-TTNT/blob/main/2001230540_Tuan04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#---2010230540--
#NguyenThanhNam
import random

# --- 1. Cấu hình Dữ liệu và Tham số ---
# Giả lập số lượng tài nguyên
NUM_LOP = 3     # Số lượng lớp học (Lớp 1, 2, 3)
NUM_GIAO_VIEN = 3 # Số lượng giáo viên (GV A, B, C)

# Các tiết học cần xếp lịch: (ID Tiết, Giáo Viên, Lớp)
# Mỗi phần tử là một "sự kiện" cần được gán một khe thời gian (Slot).
TIET_HOC = [
    (1, 'GV A', 1), (2, 'GV A', 2), (3, 'GV B', 3), (4, 'GV B', 1),
    (5, 'GV C', 2), (6, 'GV C', 3), (7, 'GV A', 3), (8, 'GV B', 2),
    (9, 'GV C', 1), (10, 'GV A', 1), (11, 'GV B', 3), (12, 'GV C', 2)
]
SO_LUONG_TIET_HOC = len(TIET_HOC) # Độ dài của nhiễm sắc thể
SO_KHE_THOI_GIAN = 5 # Số lượng khe thời gian có sẵn (Slots): 0, 1, 2, 3, 4

# --- 2. Tham số Giải thuật Di truyền (GA) ---
KICH_THUOC_QUAN_THE = 50 # Số lượng cá thể (lịch trình) trong mỗi thế hệ.
SO_THE_HE = 100 # Số lần lặp tối đa của quá trình tiến hóa.
TY_LE_LAI_GHEP = 0.8 # Xác suất để cha mẹ tiến hành lai ghép.
TY_LE_DOT_BIEN = 0.1 # Xác suất để một gen (Slot) bị thay đổi ngẫu nhiên.

# --- 3. Khởi tạo Quần thể ---
def khoi_tao_quan_the():
    """Tạo quần thể ban đầu gồm các lịch trình ngẫu nhiên."""
    quan_the = []
    for _ in range(KICH_THUOC_QUAN_THE):
        # Nhiễm sắc thể: Danh sách các Slot ngẫu nhiên được gán cho từng Tiết học.
        nhiem_sac_the = [random.randrange(SO_KHE_THOI_GIAN) for _ in range(SO_LUONG_TIET_HOC)]
        quan_the.append(nhiem_sac_the)
    return quan_the

# --- 4. Hàm Thích nghi (Fitness) ---
def tinh_diem_thich_nghi(lich_trinh):
    """Đánh giá chất lượng lịch trình bằng cách đếm tổng số xung đột."""
    xung_dot = 0

    # 1. Kiểm tra Xung đột Cứng (Giáo viên và Lớp học)
    for i in range(SO_LUONG_TIET_HOC):
        for j in range(i + 1, SO_LUONG_TIET_HOC):
            tiet_i = TIET_HOC[i]
            tiet_j = TIET_HOC[j]

            slot_i = lich_trinh[i]
            slot_j = lich_trinh[j]

            # Xung đột chỉ xảy ra nếu hai tiết cùng Slot
            if slot_i == slot_j:
                gv_i, lop_i = tiet_i[1], tiet_i[2]
                gv_j, lop_j = tiet_j[1], tiet_j[2]

                # Xung đột Giáo viên
                if gv_i == gv_j:
                    xung_dot += 1

                # Xung đột Lớp học
                if lop_i == lop_j:
                    xung_dot += 1

    # 2. Kiểm tra Xung đột Mềm (Ví dụ: Tránh Slot cuối cho tiết quan trọng)
    for i in range(SO_LUONG_TIET_HOC):
        # Giả định Tiết 10, 11, 12 là quan trọng và Slot cuối là SO_KHE_THOI_GIAN - 1
        if TIET_HOC[i][0] in [10, 11, 12] and lich_trinh[i] == SO_KHE_THOI_GIAN - 1:
            xung_dot += 0.5 # Phạt nhẹ hơn

    # Fitness: Tối đa hóa 1 / (1 + Xung_đột).
    return 1 / (1 + xung_dot), xung_dot

# --- 5. Chọn lọc (Selection) - Tournament Selection ---
def chon_loc(quan_the):
    """Chọn cá thể tốt nhất từ một nhóm ngẫu nhiên (Tournament Selection)."""
    kich_thuoc_giai_dau = 5
    nguoi_tham_gia = random.sample(quan_the, kich_thuoc_giai_dau)
    # Sắp xếp và trả về cá thể có fitness cao nhất
    nguoi_tham_gia.sort(key=lambda x: tinh_diem_thich_nghi(x)[0], reverse=True)
    return nguoi_tham_gia[0]

# --- 6. Lai ghép (Crossover) - One-Point Crossover ---
def lai_ghep(cha, me):
    """Kết hợp gen từ hai cha mẹ tại một điểm cắt ngẫu nhiên."""
    if random.random() < TY_LE_LAI_GHEP:
        diem_lai_ghep = random.randrange(1, SO_LUONG_TIET_HOC - 1)
        # Tạo con cái
        con1 = cha[:diem_lai_ghep] + me[diem_lai_ghep:]
        con2 = me[:diem_lai_ghep] + cha[diem_lai_ghep:]
        return con1, con2
    return cha, me # Không lai ghép, trả về cha mẹ

# --- 7. Đột biến (Mutation) ---
def dot_bien(nhiem_sac_the):
    """Thay đổi ngẫu nhiên một gen (Slot) với xác suất thấp."""
    for i in range(SO_LUONG_TIET_HOC):
        if random.random() < TY_LE_DOT_BIEN:
            # Thay đổi Slot tại vị trí i bằng một Slot ngẫu nhiên mới
            nhiem_sac_the[i] = random.randrange(SO_KHE_THOI_GIAN)
    return nhiem_sac_the

# --- 8. Hàm Chạy Chính (Main Loop) ---
def thuc_hien_giai_thuat_di_truyen():
    """Điều khiển vòng lặp chính của Giải thuật Di truyền."""
    quan_the = khoi_tao_quan_the()
    lich_trinh_tot_nhat = None
    fitness_tot_nhat = -1

    print(f"--- Bắt đầu Giải thuật Di truyền ({SO_THE_HE} Thế hệ) ---")

    for the_he in range(SO_THE_HE):
        # Đánh giá Fitness và sắp xếp quần thể
        quan_the_danh_gia = [(tinh_diem_thich_nghi(c), c) for c in quan_the]
        quan_the_danh_gia.sort(key=lambda x: x[0][0], reverse=True)

        fitness_hien_tai, xung_dot_hien_tai = quan_the_danh_gia[0][0]

        # Cập nhật kết quả tốt nhất toàn cục
        if fitness_hien_tai > fitness_tot_nhat:
            fitness_tot_nhat = fitness_hien_tai
            lich_trinh_tot_nhat = quan_the_danh_gia[0][1]

        if the_he % 10 == 0 or the_he == SO_THE_HE - 1:
            print(f"Thế hệ {the_he:3d}: Fitness tốt nhất = {fitness_tot_nhat:.4f} (Xung đột: {xung_dot_hien_tai})")

        # Tiêu chí dừng sớm
        if xung_dot_hien_tai == 0:
            print(f"\nlịch trình không xung đột ở Thế hệ {the_he}!")
            break

        # Tạo quần thể mới
        quan_the_moi = []
        # Elitism: Giữ lại cá thể tốt nhất
        quan_the_moi.append(quan_the_danh_gia[0][1])

        while len(quan_the_moi) < KICH_THUOC_QUAN_THE:
            # Chọn lọc cha mẹ
            cha = chon_loc(quan_the)
            me = chon_loc(quan_the)

            # Lai ghép
            con1, con2 = lai_ghep(cha[:], me[:])

            # Đột biến và thêm vào quần thể mới
            quan_the_moi.append(dot_bien(con1))
            if len(quan_the_moi) < KICH_THUOC_QUAN_THE:
                quan_the_moi.append(dot_bien(con2))

        quan_the = quan_the_moi

    # --- Kết quả cuối cùng ---
    print("\n--- KẾT QUẢ CUỐI CÙNG ---")
    final_fitness, final_conflicts = tinh_diem_thich_nghi(lich_trinh_tot_nhat)
    print(f"Fitness cuối cùng: {final_fitness:.4f}")
    print(f"Tổng số xung đột: {final_conflicts}")

    if lich_trinh_tot_nhat:
        print("\nChi tiết Lịch trình (Tiết ID: Slot):")
        # In ra lịch trình tối ưu được tìm thấy
        for i, slot in enumerate(lich_trinh_tot_nhat):
            tiet_id, gv, lop = TIET_HOC[i]
            print(f"Tiết {tiet_id} (GV {gv}, Lớp {lop}): Slot {slot}")

    return lich_trinh_tot_nhat

if __name__ == "__main__":
    thuc_hien_giai_thuat_di_truyen()

--- Bắt đầu Giải thuật Di truyền (100 Thế hệ) ---
Thế hệ   0: Fitness tốt nhất = 0.2500 (Xung đột: 3)

lịch trình không xung đột ở Thế hệ 5!

--- KẾT QUẢ CUỐI CÙNG ---
Fitness cuối cùng: 1.0000
Tổng số xung đột: 0

Chi tiết Lịch trình (Tiết ID: Slot):
Tiết 1 (GV GV A, Lớp 1): Slot 4
Tiết 2 (GV GV A, Lớp 2): Slot 2
Tiết 3 (GV GV B, Lớp 3): Slot 2
Tiết 4 (GV GV B, Lớp 1): Slot 1
Tiết 5 (GV GV C, Lớp 2): Slot 1
Tiết 6 (GV GV C, Lớp 3): Slot 4
Tiết 7 (GV GV A, Lớp 3): Slot 1
Tiết 8 (GV GV B, Lớp 2): Slot 3
Tiết 9 (GV GV C, Lớp 1): Slot 3
Tiết 10 (GV GV A, Lớp 1): Slot 0
Tiết 11 (GV GV B, Lớp 3): Slot 0
Tiết 12 (GV GV C, Lớp 2): Slot 0
